# [Expressions: Aggregation](https://docs.pola.rs/user-guide/expressions/aggregation/)

In [3]:
import polars as pl

url = "hf://datasets/nameexhaustion/polars-docs/legislators-historical.csv"

schema_overrides = {
    "first_name": pl.Categorical,
    "gender": pl.Categorical,
    "type": pl.Categorical,
    "state": pl.Categorical,
    "party": pl.Categorical,
}

dataset = (
    pl.read_csv(url, schema_overrides=schema_overrides)
    .with_columns(pl.col("first", "middle", "last").name.suffix("_name"))
    .with_columns(pl.col("birthday").str.to_date(strict=False)) # You can do multiple `with_columns` in one go!
)
dataset

bioguide,govtrack,icpsr,wikipedia,house_history,wikidata,google_entity_id,ballotpedia,first,middle,last,nickname,suffix,birthday,gender,type,start,end,state,class,party,district,first_name,middle_name,last_name
str,i64,i64,str,i64,str,str,str,str,str,str,str,str,date,cat,cat,str,str,cat,i64,cat,i64,str,str,str
"""B000226""",401222,507,"""Richard Bassett (Delaware poli…",null,"""Q518823""","""kg:/m/02pz46""",null,"""Richard""",null,"""Bassett""",null,null,1745-04-02,"""M""","""sen""","""1789-03-04""","""1793-03-03""","""DE""",2,"""Anti-Administration""",null,"""Richard""",null,"""Bassett"""
"""B000546""",401521,786,"""Theodorick Bland (congressman)""",9479,"""Q1749152""","""kg:/m/033mf4""",null,"""Theodorick""",null,"""Bland""",null,null,1742-03-21,"""M""","""rep""","""1789-03-04""","""1791-03-03""","""VA""",null,null,9,"""Theodorick""",null,"""Bland"""
"""B001086""",402032,1260,"""Aedanus Burke""",10177,"""Q380504""","""kg:/m/03yccv""",null,"""Aedanus""",null,"""Burke""",null,null,1743-06-16,"""M""","""rep""","""1789-03-04""","""1791-03-03""","""SC""",null,null,2,"""Aedanus""",null,"""Burke"""
"""C000187""",402334,1538,"""Daniel Carroll""",10687,"""Q674371""","""kg:/m/02q22c""",null,"""Daniel""",null,"""Carroll""",null,null,1730-07-22,"""M""","""rep""","""1789-03-04""","""1791-03-03""","""MD""",null,null,6,"""Daniel""",null,"""Carroll"""
"""C000538""",402671,1859,"""George Clymer""",11120,"""Q708913""","""kg:/m/01mpsj""",null,"""George""",null,"""Clymer""",null,null,1739-03-16,"""M""","""rep""","""1789-03-04""","""1791-03-03""","""PA""",null,null,-1,"""George""",null,"""Clymer"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""L000605""",456964,null,"""Erica Lee Carter""",null,"""Q130982422""",null,"""Erica Lee Carter""","""Erica""",null,"""Lee Carter""",null,null,1980-02-02,"""F""","""rep""","""2024-11-05""","""2025-01-03""","""TX""",null,"""Democrat""",18,"""Erica""",null,"""Lee Carter"""
"""V000137""",456876,null,"""JD Vance""",null,"""Q28935729""","""kg:/g/11c6v_wj1r""","""J.D. Vance""","""James David""",null,"""Vance""","""J.D.""",null,1984-08-02,"""M""","""sen""","""2023-01-03""","""2025-01-09""","""OH""",3,"""Republican""",null,"""James David""",null,"""Vance"""
"""R000595""",412491,41102,"""Marco Rubio""",null,"""Q324546""","""kg:/m/0dpr5f""","""Marco Rubio""","""Marco""",null,"""Rubio""",null,null,1971-05-28,"""M""","""sen""","""2011-01-05""","""2017-01-03""","""FL""",3,"""Republican""",null,"""Marco""",null,"""Rubio"""


## Basic aggregations

In [8]:
q = (
    dataset.lazy()
    .group_by("first_name")
    .agg(
        pl.len(),
        pl.col("gender"),
        pl.first("last_name")
    )
    .sort("len", descending=True)
)

df = q.collect()
print(df)

shape: (1_603, 4)
┌────────────┬──────┬───────────────────┬────────────┐
│ first_name ┆ len  ┆ gender            ┆ last_name  │
│ ---        ┆ ---  ┆ ---               ┆ ---        │
│ str        ┆ u32  ┆ list[cat]         ┆ str        │
╞════════════╪══════╪═══════════════════╪════════════╡
│ John       ┆ 4227 ┆ ["M", "M", … "M"] ┆ Walker     │
│ William    ┆ 3309 ┆ ["M", "M", … "M"] ┆ Few        │
│ James      ┆ 2414 ┆ ["M", "M", … "M"] ┆ Armstrong  │
│ Charles    ┆ 1514 ┆ ["M", "M", … "M"] ┆ Carroll    │
│ Thomas     ┆ 1502 ┆ ["M", "M", … "M"] ┆ Tucker     │
│ …          ┆ …    ┆ …                 ┆ …          │
│ Maryon     ┆ 1    ┆ ["F"]             ┆ Allen      │
│ Gershom    ┆ 1    ┆ ["M"]             ┆ Powers     │
│ Kerry      ┆ 1    ┆ ["M"]             ┆ Bentivolio │
│ Michel     ┆ 1    ┆ ["M"]             ┆ Vidal      │
│ Lansing    ┆ 1    ┆ ["M"]             ┆ Stout      │
└────────────┴──────┴───────────────────┴────────────┘


## conditionals

In [15]:
q = (
    dataset.lazy()
    .group_by("state")
    .agg(
        (pl.col("party") == "Anti-Administration").sum().alias("anti"),
        (pl.col("party") == "Pro-Administration").sum().alias("pro")
    )
    .sort("pro", descending=True)
    .limit(5)
    )
df = q.collect()
df

state,anti,pro
cat,u32,u32
"""CT""",0,5
"""DE""",1,3
"""NJ""",0,3
"""NC""",2,2
"""MA""",0,2


## Filtering

In [ ]:
from datetime import date

def compute_age():
    return date.today().year - pl.col("birthday").dt.year()

def avg_birthday(gender: str) -> pl.Expr:
    return(
        compute_age()
        .filter(pl.col("gender") == gender)
        .mean()
        .alias(f"avg {gender} birthday")
    )

q = (
    dataset.lazy()
    .group_by("state")
    .agg(
        avg_birthday("M"),
        avg_birthday("F"),
        (pl.col("gender") == "M").sum().alias("# male"),
        (pl.col("gender") == "F").sum().alias("# female"),
    )
    .sort("state")
    .limit(5)
)

df = q.collect()
print(df)


shape: (5, 5)
┌───────┬────────────────┬────────────────┬────────┬──────────┐
│ state ┆ avg M birthday ┆ avg F birthday ┆ # male ┆ # female │
│ ---   ┆ ---            ┆ ---            ┆ ---    ┆ ---      │
│ cat   ┆ f64            ┆ f64            ┆ u32    ┆ u32      │
╞═══════╪════════════════╪════════════════╪════════╪══════════╡
│ DE    ┆ 176.45933      ┆ null           ┆ 223    ┆ 0        │
│ VA    ┆ 182.590175     ┆ 63.058824      ┆ 1599   ┆ 17       │
│ SC    ┆ 170.103666     ┆ 114.428571     ┆ 876    ┆ 7        │
│ MD    ┆ 174.803419     ┆ 93.826087      ┆ 881    ┆ 46       │
│ PA    ┆ 166.008592     ┆ 91.636364      ┆ 3205   ┆ 22       │
└───────┴────────────────┴────────────────┴────────┴──────────┘


## nested grouping

In [238]:
# sorting is a bit random, with every run, it sorts different when only sorting on `count`
q = (
    dataset.lazy()
    .group_by("state", "party")
    .agg(pl.len().alias("count"))
    .filter(
        (pl.col("party") == "Anti-Administration") | 
        (pl.col("party") == "Pro-Administration")
    )
    .sort(["count", "state"], descending=True)
    .limit(5)
)

df = q.collect()
df

state,party,count
cat,cat,u32
"""CT""","""Pro-Administration""",5
"""VA""","""Anti-Administration""",5
"""NJ""","""Pro-Administration""",3
"""PA""","""Anti-Administration""",3
"""DE""","""Pro-Administration""",3


In [247]:
q = (
    dataset.lazy()
    .group_by("state", "gender")
    .agg(
        compute_age().mean().alias("avg birthday"), # user defined function
        pl.len().alias("#")
    )
    .sort("#", descending=True)
    .limit(5)
)


df = q.collect()
print(df)

shape: (5, 4)
┌───────┬────────┬──────────────┬──────┐
│ state ┆ gender ┆ avg birthday ┆ #    │
│ ---   ┆ ---    ┆ ---          ┆ ---  │
│ cat   ┆ cat    ┆ f64          ┆ u32  │
╞═══════╪════════╪══════════════╪══════╡
│ NY    ┆ M      ┆ 164.204634   ┆ 3965 │
│ PA    ┆ M      ┆ 166.008592   ┆ 3205 │
│ OH    ┆ M      ┆ 156.579961   ┆ 2142 │
│ IL    ┆ M      ┆ 145.069482   ┆ 1895 │
│ CA    ┆ M      ┆ 114.400464   ┆ 1725 │
└───────┴────────┴──────────────┴──────┘


## sorting

In [298]:
def get_name() -> pl.Expr:
    return pl.col("first_name") + pl.lit(" ") + pl.col("last_name")

q = (
    dataset.lazy()
    .sort("birthday", descending=True)
    .group_by("state")
    .agg(
        get_name().first().alias("youngest"),
        get_name().last().alias("oldest")
    )
    .sort("state", descending=True)
    .limit(5)
)

df = q.collect()
df

state,youngest,oldest
cat,str,str
"""AS""","""Eni Faleomavaega""","""Fofó Sunia"""
"""VI""","""Donna Christensen""","""Melvin Evans"""
"""GU""","""Michael San Nicolas""","""Antonio Won Pat"""
"""MP""","""Gregorio Sablan""","""Gregorio Sablan"""
"""PI""","""Carlos Romulo""","""Pablo Ocampo"""


but we can also sort before we do the aggregation

In [ ]:
q = (
    dataset.lazy()
    .sort("birthday", descending=True)
    .group_by("state")
    .agg(
        get_name().first().alias("youngest"),
        get_name().last().alias("oldest"),
        get_name().sort().first().alias("alphabetically first"),
    )
    .sort("state")
    .limit(5)
)

df = q.collect()
df

state,youngest,oldest,alphabeticaly first
cat,str,str,str
"""DE""","""Samuel White""","""George Read""","""Albert Polk"""
"""VA""","""William Grayson""","""Robert Rutherford""","""A. McEachin"""
"""SC""","""Ralph Izard""","""Thomas Sumter""","""Abraham Nott"""
"""MD""","""Benjamin Contee""","""William Smith""","""Albert Blakeney"""
"""PA""","""Thomas Fitzsimons""","""Israel Jacobs""","""Aaron Kreider"""


It's possible to sort a column with the order induced by another column: `.sort_by(get_name())`

In [330]:
q = (
    dataset.lazy()
    .sort("birthday", descending=True)
    .group_by("state")
    .agg(
        get_name().first().alias("youngest"),
        get_name().last().alias("oldest"),
        get_name().sort().first().alias("alphabetically first"),
        pl.col("gender").sort_by(get_name()).first(),
    )
    .sort("state")
    .limit(5)
)

df = q.collect()
df

state,youngest,oldest,alphabetically first,gender
cat,str,str,str,cat
"""DE""","""Samuel White""","""George Read""","""Albert Polk""","""M"""
"""VA""","""William Grayson""","""Robert Rutherford""","""A. McEachin""","""M"""
"""SC""","""Ralph Izard""","""Thomas Sumter""","""Abraham Nott""","""M"""
"""MD""","""Benjamin Contee""","""William Smith""","""Albert Blakeney""","""M"""
"""PA""","""Thomas Fitzsimons""","""Israel Jacobs""","""Aaron Kreider""","""M"""


note: using lambda's can destroy parallelization in polars.